# Working with OptiGraphs

In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__,".."))
push!(LOAD_PATH,joinpath(@__DIR__,"../.."))
using Plasmo
using Ipopt
using KaHyPar

  Activating project at `~/git/Plasmo.jl/examples`
[ Info: Precompiling Plasmo [d3f7391f-f14a-50cc-bbe4-76a32d1bad3c]


## Create the OptiGraph
Here we create an optigraph that represents a dynamic optimization problem. 

In [2]:
N = 100
x_reference = ones(N)

graph = OptiGraph()
@optinode(graph, nodes[1:N])
@nodevariables(nodes, x >= 0)
@nodevariables(nodes, u)
for i=1:N
    @objective(nodes[i], Min, (nodes[i][:x] - x_reference[i])^2 + nodes[i][:u]^2)
end
for j in 1:N-1
    @linkconstraint(graph, nodes[j+1][:x] == nodes[j][:x] + nodes[j][:u])
end

# fix initial condition
# NOTE: need to delete old bound
fix(nodes[1][:x], 5.0; force=true)

graph

An OptiGraph:    g##225
                 #local elements  #total elements
--------------------------------------------------
          Nodes:       100              100
          Edges:        99               99
      Subgraphs:         0                0
      Variables:       200              200
    Constraints:       199              199


## Partition the OptiGraph
An optigraph permits a natural hypergraph representation. Plasmo.jl uses such graph representations to allow partitioning of an optigraph into a new optigraph with subgraphs that correspond to the partitions. Here we create a hypergraph projection from an optigraph, we use KaHyPar to partition the result hypergraph, and we assemble a new optigraph from the partition solution.

### Create a Hypergraph Projection
Here we create a hypergraph object that maps optinode and optiedge elements to hypergraph elements. While the hypergraph projection is the most natural Plasmo.jl supports a few different projections that can be used to partition an optimization problem and produce fundamentally different partitions. See the full documentation for the available projections you can partition with.

In [5]:
projection = hyper_projection(graph)
projection

Graph Projection: Plasmo.HyperGraphProjectionType()

### Minimize the edge cut subject to balance constraints
It is also possible to minimize the *connectivity* using KaHyPar. This is a common metric to minimize communication.

In [6]:
n_partitions = 20
partition_vector = KaHyPar.partition(projection, n_partitions, configuration = :edge_cut) #configuration=:connectivity
partition_vector

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
+                    _  __     _   _       ____                               + 
+                   | |/ /__ _| | | |_   _|  _ \ __ _ _ __                    + 
+                   | ' // _` | |_| | | | | |_) / _` | '__|                   + 
+                   | . \ (_| |  _  | |_| |  __/ (_| | |                      + 
+                   |_|\_\__,_|_| |_|\__, |_|   \__,_|_|                      + 
+                                    |___/                                    + 
+                 Karlsruhe Hypergraph Partitioning Framework                 + 
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
*******************************************************************************
*                            Partitioning Context                             *
*******************************************************************************
Partitioning Parameters:
  Hype

100-element Vector{Int64}:
 10
 10
 10
 10
 10
 11
 11
 11
 11
 11
 12
 12
 12
  ⋮
  7
  7
  9
  9
  9
  9
  9
  8
  8
  8
  8
  8

## Assemble New OptiGraph from Partition Result

In [7]:
partition = Partition(projection, partition_vector)
new_graph = assemble_optigraph(partition; name=:assembled_graph)

An OptiGraph: assembled_graph
                 #local elements  #total elements
--------------------------------------------------
          Nodes:         0              100
          Edges:        19               99
      Subgraphs:        20               20
      Variables:         0              200
    Constraints:        19              199


We now have a new optigraph with subgraphs that correspond to the partitions. It is now possible to do things like visualize the structure or implement custom solution algorithms where the subgraphs inform the subproblems. While the Plasmo.jl ecosystem currently lacks standard solver interfaces for optigraphs, the tools and data structures now exist to develop such solvers. This is an area of current work; we are happy if you want to get involved!  